**Import packages**

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import re
import time

**Connect to Google Drive and upload files directly**

Google Colab has a limited RAM of 12 GB, split file into smaller ones and upload onto Google Drive folder.

Input the file path next to `csvFile`.

Use this program to split easily: [COLAB LINK](https://colab.research.google.com/drive/1RUc-R-4PDlw8JZ7Hiu2RT2YcDvZPlB44?usp=sharing)

In [2]:
from google.colab import drive

drive.mount('/content/drive')

csvFile = '/content/drive/My Drive/Mutual Fund Research Project/split_files/2021-10_second_half.csv'

data = pd.read_csv(csvFile)


Mounted at /content/drive


**Fetch XML links for NPORT-P**

`batch_size = 10`

This determines how the code process is displayed but doesn't affect the program itself, feel free to make this number higher if fewer updates are needed and vice versa.

Increase `pause_duration` if met with errors such as `Access Denied` from SEC. Usually would show up as `Failed to retrieve webpage` error.

In [3]:
all_links = []
total_links = 0
batch_size = 100  # Progress update after N links, feel free to change if you want a higher or lower number
delay_after_rows = 500  # Add delay to not get denied access from SEC
pause_duration = 60

for index, x in enumerate(data['File Name'], start=1):
    webpage_url = x
    headers = {"User-Agent": "starlin@bc.edu"}
    response = requests.get(x, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        links = soup.find_all('a')
        include_pattern = re.compile(r'^/Archives/edgar/data/\d+/\d+/primary_doc\.xml$')
        xml_links = [link.get('href') for link in links if include_pattern.match(link.get('href'))]
        base_url = 'https://www.sec.gov'
        xml_links = [url if url.startswith('http') else base_url + url for url in xml_links]

        for xml_link in xml_links:
            all_links.append(xml_link)
            total_links += 1

            if total_links % batch_size == 0 or total_links == len(data['File Name']):
                print(f"Processed link {total_links}/{len(data['File Name'])}")

            # Check if total_links is a multiple of delay_after_rows
            if total_links % delay_after_rows == 0:
                print(f"Pausing for {pause_duration} seconds after processing row {index}...")
                time.sleep(pause_duration)

    else:
        print(f"Failed to retrieve webpage: {webpage_url}, Status Code: {response.status_code}")

print("All XML links collected.")

Processed link 100/1182
Processed link 200/1182
Processed link 300/1182
Processed link 400/1182
Processed link 500/1182
Pausing for 60 seconds after processing row 500...
Processed link 600/1182
Processed link 700/1182
Processed link 800/1182
Processed link 900/1182
Processed link 1000/1182
Pausing for 60 seconds after processing row 1000...
Processed link 1100/1182
Processed link 1182/1182
All XML links collected.


In [4]:
data['XML Link'] = ''
if len(all_links) == len(data):
    data['XML Link'] = all_links
else:
    print(f'Warning: The number of XML links ({len(all_links)}) does not match the number of rows in the DataFrame ({len(data)}).')

**Fetch data for all securities**

In [5]:
def get_text_or_default(elements, index, default='N/A'):
    return elements[index].text if len(elements) > index else default

In [ ]:
all_data = []
total_rows = len(data)
batch_size = 10  # Number of rows processed before printing an update
batches_processed = 0
pause_after_batches = 100  # Pause after every 100 batches of securities fetched
pause_duration = 30  # Pause duration in seconds

for index, row in data.iterrows():
    headers = {"User-Agent": "starlin@bc.edu"}
    xml_link = row['XML Link']

    print(f"Processing row {index + 1}/{total_rows} - XML Link: {xml_link}")

    n_port_file = requests.get(xml_link, headers=headers)

    if n_port_file.status_code == 200:
        soup = BeautifulSoup(n_port_file.content, 'xml')

        # Extract fund-level information from the root element
        fund_name = soup.find('regName').text if soup.find('regName') else 'N/A'
        cik = soup.find('cik').text if soup.find('cik') else 'N/A'
        small_fund = soup.find('seriesName').text if soup.find('seriesName') else 'N/A'
        series_id = soup.find('seriesId').text if soup.find('seriesId') else 'N/A'
        report_date = soup.find('repPdDate').text if soup.find('repPdDate') else 'N/A'

        # Find all invstOrSec elements
        invstOrSecs = soup.find_all('invstOrSec')

        for invstOrSec in invstOrSecs:
            # Extract lists from invstOrSec
            security_names = [name.text.strip() for name in invstOrSec.find_all('name')]
            is_private_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('isRestrictedSec')]
            long_short_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('payoffProfile')]
            share_class_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('assetCat')]
            fair_value_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('fairValLevel')]
            val_usd_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('valUSD')]
            shares_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('balance')]
            units_list = [item.text.strip() if item else 'N/A' for item in invstOrSec.find_all('units')]

            max_len = max(len(security_names), len(is_private_list), len(long_short_list),
                          len(share_class_list), len(fair_value_list), len(val_usd_list),
                          len(shares_list), len(units_list))

            for i in range(max_len):
                row_data = [
                    row['Form Type'],
                    row['Company Name'],
                    row['CIK'],
                    row['Date Filed'],
                    row['File Name'],
                    row['XML Link'],
                    fund_name,
                    cik,
                    small_fund,
                    series_id,
                    report_date,
                    security_names[i] if i < len(security_names) else 'N/A',
                    is_private_list[i] if i < len(is_private_list) else 'N/A',
                    long_short_list[i] if i < len(long_short_list) else 'N/A',
                    share_class_list[i] if i < len(share_class_list) else 'N/A',
                    fair_value_list[i] if i < len(fair_value_list) else 'N/A',
                    val_usd_list[i] if i < len(val_usd_list) else 'N/A',
                    shares_list[i] if i < len(shares_list) else 'N/A',
                    units_list[i] if i < len(units_list) else 'N/A',
                ]
                all_data.append(row_data)

                print(f"Processed row {index + 1}/{total_rows}, security {i + 1} of {max_len}")

        batches_processed += 1

        # Pause after every 100 batches processed
        if batches_processed % pause_after_batches == 0:
            print(f"Pausing for {pause_duration} seconds to prevent access denial...")
            time.sleep(pause_duration)

    else:
        row_data = [
            row['Form Type'],
            row['Company Name'],
            row['CIK'],
            row['Date Filed'],
            row['File Name'],
            row['XML Link'],
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A',
            'N/A'
        ]
        all_data.append(row_data)

        print(f"Failed to retrieve XML for row {index + 1}/{total_rows}: Status Code {n_port_file.status_code}")

print("Data processing complete.")

Streaming output truncated to the last 5000 lines.
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, security 1 of 1
Processed row 495/1182, secur

In [ ]:
columns = [
    'Form Type', 'Company Name', 'CIK', 'Date Filed', 'File Name', 'XML Link',
    'Fund Name', 'CIK', 'Small Fund', 'Series ID', 'Report Date', 'Security Name',
    'Is Private', 'Long Short', 'Share Class', 'Fair Value', 'Val USD', 'Shares', 'Units'
]
all_data_df = pd.DataFrame(all_data, columns=columns)

**Filter to include only private securities**

In [ ]:
private_df = all_data_df[all_data_df['Is Private'] == 'Y']

**Convert into Excel sheet and save to Drive**\
Change the path next to `save_path` to save the ouput Excel file into Google Drive.\
Edit `base_name` if you want to change the file name.

In [ ]:
import openpyxl
import os

# Define the base name and new Excel file path
base_name = os.path.splitext(os.path.basename(csvFile))[0]
excel_file = f"{base_name}.xlsx"

# Path to save the new Excel file in Google Drive
save_path = f"/content/drive/My Drive/Mutual Fund Research Project/Excel outputs/{excel_file}"

# Save the DataFrame to the new Excel file in Google Drive
private_df.to_excel(save_path, index=False)

print(f"File saved to {save_path}")